In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Bidirectional, GRU, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Constants
data_dir = r"E:\Research\ML - Machine Learning\Music Genre Classification\Data\spectrogram_from_mahin"  # Path to your dataset containing subfolders for each genre
input_shape = (288, 432, 3)  # Adjust the input shape based on your spectrogram size
batch_size = 16
epochs = 30

# Data Preprocessing using ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
)

# Define the ResNet model
def build_resnet(input_shape, num_classes):
    base_model = tf.keras.applications.ResNet50(
        include_top=False, weights='imagenet', input_shape=input_shape
    )
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    return model

# Define the Bi-GRU model
def build_bigru(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    # Reshape the features to include a time dimension
    x = tf.keras.layers.Reshape((1, -1))(x)
    x = Bidirectional(GRU(128))(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output)
    return model

# Build the combined model
num_classes = len(train_generator.class_indices)
resnet_model = build_resnet(input_shape, num_classes)
bigru_model = build_bigru(input_shape, num_classes)

combined_input = Input(shape=input_shape)
resnet_features = resnet_model(combined_input)
bigru_features = bigru_model(combined_input)
combined_features = tf.keras.layers.Concatenate(axis=-1)([resnet_features, bigru_features])
output = Dense(num_classes, activation='softmax')(combined_features)
combined_model = Model(inputs=combined_input, outputs=output)

# Compile the model
combined_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# Train the model
history = combined_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    verbose=1,
)

# Evaluate the model on test data
test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',  # You can change this to 'test' if you have a separate test folder
)

y_true = test_generator.classes
y_pred = combined_model.predict(test_generator)

# Convert predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Classification report and accuracy
classification_rep = classification_report(y_true, y_pred_labels, target_names=train_generator.class_indices.keys())
accuracy = accuracy_score(y_true, y_pred_labels)

# Plot the training history
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Print classification report and accuracy
print("Classification Report:\n", classification_rep)
print("Accuracy:", accuracy)


Found 6168 images belonging to 8 classes.
Found 1541 images belonging to 8 classes.
Epoch 1/30


ResourceExhaustedError: Graph execution error:

Detected at node 'sub_34' defined at (most recent call last):
    File "C:\Users\Asus\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Asus\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Asus\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Asus\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\Asus\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\Asus\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\Asus\anaconda3\lib\asyncio\base_events.py", line 603, in run_forever
      self._run_once()
    File "C:\Users\Asus